In [1]:
import torch
from onnx_tf.backend import prepare
import onnx
import tensorflow as tf

In [2]:
loaded = torch.jit.load('7x7/best.pt')
print(loaded)

RecursiveScriptModule(
  original_name=DualNet
  (conv1): RecursiveScriptModule(original_name=Conv2d)
  (bn1): RecursiveScriptModule(original_name=BatchNorm2d)
  (relu): RecursiveScriptModule(original_name=ReLU)
  (layers): RecursiveScriptModule(
    original_name=Sequential
    (0): RecursiveScriptModule(
      original_name=BasicBlock
      (conv1): RecursiveScriptModule(original_name=Conv2d)
      (bn1): RecursiveScriptModule(original_name=BatchNorm2d)
      (relu): RecursiveScriptModule(original_name=ReLU)
      (conv2): RecursiveScriptModule(original_name=Conv2d)
      (bn2): RecursiveScriptModule(original_name=BatchNorm2d)
      (downsample): RecursiveScriptModule(original_name=Sequential)
    )
    (1): RecursiveScriptModule(
      original_name=BasicBlock
      (conv1): RecursiveScriptModule(original_name=Conv2d)
      (bn1): RecursiveScriptModule(original_name=BatchNorm2d)
      (relu): RecursiveScriptModule(original_name=ReLU)
      (conv2): RecursiveScriptModule(original_nam

In [5]:
loaded.eval()
bisze = 7
dummy_input = torch.randn((16, 12, bisze, bisze))
torch.onnx.export(loaded, dummy_input, "dualnet.onnx", example_outputs=torch.rand((16, bisze*bisze+2)), verbose=True)

C:\Users\sen\anaconda3\envs\py38\lib\site-packages\torch\onnx\utils.py:341: UserWarning: Model has no forward function
  warnings.warn("Model has no forward function")
C:\Users\sen\anaconda3\envs\py38\lib\site-packages\torch\onnx\symbolic_helper.py:712: UserWarning: ONNX export mode is set to inference mode, but operator batch_norm is set to training  mode. The model will be exported in inference, as specified by the export mode.
  warnings.warn("ONNX export mode is set to " + training_mode +


graph(%x.1 : Float(16, 12, 7, 7, strides=[588, 49, 7, 1], requires_grad=0, device=cpu),
      %pi.4.bias : Float(50, strides=[1], requires_grad=0, device=cuda:0),
      %pi.4.weight : Float(50, 1568, strides=[1568, 1], requires_grad=0, device=cuda:0),
      %v.4.bias : Float(32, strides=[1], requires_grad=0, device=cuda:0),
      %v.4.weight : Float(32, 147, strides=[147, 1], requires_grad=0, device=cuda:0),
      %v.6.bias : Float(1, strides=[1], requires_grad=0, device=cuda:0),
      %v.6.weight : Float(1, 32, strides=[32, 1], requires_grad=0, device=cuda:0),
      %316 : Float(32, 12, 3, 3, strides=[108, 9, 3, 1], requires_grad=0, device=cuda:0),
      %317 : Float(32, strides=[1], requires_grad=0, device=cuda:0),
      %319 : Float(32, 32, 3, 3, strides=[288, 9, 3, 1], requires_grad=0, device=cuda:0),
      %320 : Float(32, strides=[1], requires_grad=0, device=cuda:0),
      %322 : Float(32, 32, 3, 3, strides=[288, 9, 3, 1], requires_grad=0, device=cuda:0),
      %323 : Float(32, s

In [6]:
tf.autograph.experimental.do_not_convert
onnx_model = onnx.load('dualnet.onnx')

tf_rep = prepare(onnx_model, device='cpu')
tf_rep.export_graph('model.pb')

INFO:tensorflow:Assets written to: model.pb\assets


In [7]:
new_model = tf.saved_model.load('model.pb')
f = new_model.signatures["serving_default"]
print(f)

<ConcreteFunction signature_wrapper(x.1) at 0x22A3DB97B20>


In [8]:
!cd train
!tensorflowjs_converter --input_format=tf_saved_model --output_node_names='output_0' model.pb js_model --quantize_uint8

�w�肳�ꂽ�p�X��������܂���B


Writing weight file js_model\model.json...
weight PartitionedCall/onnx_tf_prefix_Constant_0 with shape (4,) and dtype int64 was auto converted to the type int32
weight PartitionedCall/Where with shape (0, 1) and dtype int64 was auto converted to the type int32
weight PartitionedCall/Cast with shape (1,) and dtype int64 was auto converted to the type int32
weight PartitionedCall/GatherV2 with shape (0,) and dtype int64 was auto converted to the type int32
weight PartitionedCall/zeros with shape () and dtype int64 was auto converted to the type int32


2021-12-02 20:48:53.103689: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations:  AVX AVX2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-12-02 20:48:54.093769: I tensorflow/core/grappler/devices.cc:78] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
2021-12-02 20:48:54.094127: I tensorflow/core/grappler/clusters/single_machine.cc:356] Starting new session
2021-12-02 20:48:54.288361: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:816] Optimization results for grappler item: graph_to_optimize
2021-12-02 20:48:54.288697: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:818]   function_optimizer: Graph size after: 811 nodes (732), 761 edges (683), time = 26.733ms.
2021-12-02 20:48:54.